In [ ]:
# /// script
# requires-python = ">=3.13"
# dependencies = [
#     "httpx",
#     "pystac-client",
#     "requests",
# ]
# ///

The STAC Collection Discovery API is a collection-search-only STAC API that combines paginated search results from multiple upstream STAC APIs into a single endpoint.

## Overview

This API acts as a proxy/aggregator, allowing you to search for collections across multiple STAC catalogs simultaneously. Instead of querying each STAC API individually, you can use this service to get unified results from all configured endpoints.

### Key Features

- **Unified Collection Search**: Search across multiple STAC APIs with a single request
- **Dynamic API Selection**: Override default APIs using the `apis` parameter
- **Smart Conformance Classes**: Only returns capabilities supported by ALL queried APIs
- **Token-based Pagination**: Maintains state across all upstream APIs
- **Standard STAC Extensions**: Supports filtering, sorting, free-text search, and field selection

### Important Behavior

- **Pagination Multiplier**: The `limit` parameter applies to each upstream API, so with `limit=10` and 3 APIs, you may get up to 30 collections per page
- **Conformance Intersection**: Only collection-search features supported by ALL upstream APIs are available in the response
- **Enabled Extensions**: Limited to filter, sort, free-text search, and fields selection extensions

Let's explore how to use this API with some practical examples.

In [1]:
import httpx
import pystac_client
from requests import Request

# Assumes the docker network is running
API_BASE_URL = "http://localhost:8000"

## HTTP Requests

The following section shows how to use `httpx` to perform queries on the API.

In [2]:
# Get the landing page to see what APIs are configured
response = httpx.get(f"{API_BASE_URL}/")
landing_page = response.json()

print("API Title:", landing_page["title"])
print("Description:", landing_page["description"])
print("\nConfigured upstream APIs:")
for link in landing_page["links"]:
    if link["rel"] == "child":
        print(f"- {link['href']}")

API Title: STAC Collection Discovery API
Description: A collection-search-only STAC API that combines paginated search results from multiple upstream STAC APIs.

## API Configuration

This API has been pre-configured to search this set of upstream STAC APIs by default:

- https://stac.maap-project.org

- https://stac.eoapi.dev

- https://cmr.earthdata.nasa.gov/stac/ALL

Users can override this configuration for individual requests by providing their own list of APIs using the `apis` query parameter, either as multiple parameters (`?apis=url1&apis=url2`) or as a comma-separated string (`?apis=url1,url2`).

## Conformance Classes

The API's conformance classes are dynamically calculated based on the intersection of capabilities across all queried upstream APIs. The conformance classes returned for any given request represent only the collection-search features that are supported by ALL upstream APIs in the request, limited to the extensions enabled in this application: filter, sort, free

### Check Conformance Classes

The conformance classes tell us what capabilities are available. Remember, these are dynamically calculated based on what ALL upstream APIs support.

In [3]:
# Check what features are supported
response = httpx.get(f"{API_BASE_URL}/conformance")
conformance = response.json()

print("Supported collection-search conformance classes:")
for cls in conformance["conformsTo"]:
    if "collection-search" in cls:
        print(f"- {cls}")

Supported collection-search conformance classes:
- https://api.stacspec.org/v1.0.0-rc.1/collection-search
- https://api.stacspec.org/v1.0.0-rc.1/collection-search#free-text
- https://api.stacspec.org/v1.0.0-rc.1/collection-search#sort


The important ones for this application are the `/collection-search` conformance classes. In this example, basic collection search is available along with the `sort` and `free-text` extensions.

If we specify a specific set of upstream APIs (using the `apis` parameter), the conformance classes can change.

In [4]:
response = httpx.get(
    f"{API_BASE_URL}/conformance",
    params={
        "apis": [
            "https://stac.maap-project.org",
            "https://stac.eoapi.dev",
        ]
    },
)
conformance = response.json()

print("Supported conformance classes:")
for cls in conformance["conformsTo"]:
    if "collection-search" in cls:
        print(f"- {cls}")

Supported conformance classes:
- https://api.stacspec.org/v1.0.0-rc.1/collection-search
- https://api.stacspec.org/v1.0.0-rc.1/collection-search#fields
- https://api.stacspec.org/v1.0.0-rc.1/collection-search#filter
- https://api.stacspec.org/v1.0.0-rc.1/collection-search#free-text
- https://api.stacspec.org/v1.0.0-rc.1/collection-search#sort


By not including `https://cmr.earthdata.nasa.gov/stac/ALL` (which doesn't include the `filter` extension), our API is now capable of applying the `filter` extension to collection searches!

## Basic Collection Search

Let's start with a simple collection search across all of the pre-configured APIs.

In [13]:
# Basic collection search with a small limit
response = httpx.get(f"{API_BASE_URL}/collections?limit=2")
results = response.json()

print(f"Number of collections returned: {len(results['collections'])}")

print("\nCollection IDs:")
for collection in results["collections"]:
    print(f"- {collection['id']}")

print("\nPagination links:")
for link in results["links"]:
    if link["rel"] in ["self", "next", "previous"]:
        print(f"- {link['rel']}: {link['href']}")

Number of collections returned: 6

Collection IDs:
- ABoVE_UAVSAR_PALSAR
- AFRISAR_DLR
- copernicus-dem
- MAXAR_afghanistan_earthquake22
- latent-reserves-in-the-swiss-nfi_1.0
- ASAC_2201_HCL_0.5_1

Pagination links:
- self: http://localhost:8000/collections?limit=2
- next: http://localhost:8000/collections?token=eyJjdXJyZW50Ijp7Imh0dHBzOi8vc3RhYy5tYWFwLXByb2plY3Qub3JnIjoiaHR0cHM6Ly9zdGFjLm1hYXAtcHJvamVjdC5vcmcvY29sbGVjdGlvbnM_bGltaXQ9MiZmaWx0ZXJfbGFuZz1jcWwyLXRleHQmb2Zmc2V0PTIiLCJodHRwczovL3N0YWMuZW9hcGkuZGV2IjoiaHR0cHM6Ly9zdGFjLmVvYXBpLmRldi9jb2xsZWN0aW9ucz9saW1pdD0yJmZpbHRlcl9sYW5nPWNxbDItdGV4dCZvZmZzZXQ9MiIsImh0dHBzOi8vY21yLmVhcnRoZGF0YS5uYXNhLmdvdi9zdGFjL0FMTCI6Imh0dHBzOi8vY21yLmVhcnRoZGF0YS5uYXNhLmdvdi9zdGFjL0FMTC9jb2xsZWN0aW9ucz9maWx0ZXJfbGFuZz1jcWwyLXRleHQmbGltaXQ9MiZjdXJzb3I9ZXlKcWMyOXVJam9pVzF3aU1DNDFJR2h2ZFhJZ01TQnRJR2hqYkNCbGVIUnlZV04wYVc5dUlHUmhkR0VnWm05eUlIUm9aU0IzYVc1a2JXbHNiQ0JwYzJ4aGJtUnpJRzFoY21sdVpTQnpaV1JwYldWdWRITmNJaXhjSWtGVlgwRkJSRU5jSWl4Y0lrRlRRVU5mTWpJd01WOUlRM

Notice how with `limit=2`, we will get more than two collections total since the limit applies to each upstream API individually.

### Pagination
The initial request returned the first page of results from each upstream API and included a `next` link with the search state embedded in it. When you send a request to the `next` link the API will decode the token into a list of `next` links from the upstream APIs, send those requests, and return the results back to you.

In [14]:
next_link = next(
    (link["href"] for link in results["links"] if link["rel"] == "next"),
    None,
)

response = httpx.get(next_link)
results = response.json()

print(f"Number of collections returned in second page: {len(results['collections'])}")

print("\nCollection IDs:")
for collection in results["collections"]:
    print(f"- {collection['id']}")

print("\nPagination links:")
for link in results["links"]:
    if link["rel"] in ["self", "next", "previous"]:
        print(f"- {link['rel']}: {link['href']}")

Number of collections returned in second page: 6

Collection IDs:
- AFRISAR_DLR2
- AfriSAR_UAVSAR_Coreg_SLC
- MAXAR_BayofBengal_Cyclone_Mocha_May_23
- MAXAR_cyclone_emnati22
- urn:ogc:def:EOP:VITO:VGT_S10_1
- LGB_10m_traverse_1

Pagination links:
- self: http://localhost:8000/collections?token=eyJjdXJyZW50Ijp7Imh0dHBzOi8vc3RhYy5tYWFwLXByb2plY3Qub3JnIjoiaHR0cHM6Ly9zdGFjLm1hYXAtcHJvamVjdC5vcmcvY29sbGVjdGlvbnM_bGltaXQ9MiZmaWx0ZXJfbGFuZz1jcWwyLXRleHQmb2Zmc2V0PTIiLCJodHRwczovL3N0YWMuZW9hcGkuZGV2IjoiaHR0cHM6Ly9zdGFjLmVvYXBpLmRldi9jb2xsZWN0aW9ucz9saW1pdD0yJmZpbHRlcl9sYW5nPWNxbDItdGV4dCZvZmZzZXQ9MiIsImh0dHBzOi8vY21yLmVhcnRoZGF0YS5uYXNhLmdvdi9zdGFjL0FMTCI6Imh0dHBzOi8vY21yLmVhcnRoZGF0YS5uYXNhLmdvdi9zdGFjL0FMTC9jb2xsZWN0aW9ucz9maWx0ZXJfbGFuZz1jcWwyLXRleHQmbGltaXQ9MiZjdXJzb3I9ZXlKcWMyOXVJam9pVzF3aU1DNDFJR2h2ZFhJZ01TQnRJR2hqYkNCbGVIUnlZV04wYVc5dUlHUmhkR0VnWm05eUlIUm9aU0IzYVc1a2JXbHNiQ0JwYzJ4aGJtUnpJRzFoY21sdVpTQnpaV1JwYldWdWRITmNJaXhjSWtGVlgwRkJSRU5jSWl4Y0lrRlRRVU5mTWpJd01WOUlRMHhmTUM0MVhDSXNYQ0l4W

### Searching Specific APIs

You can override the default configured APIs by specifying which ones to query:

In [15]:
# Search only specific APIs
params = {
    "apis": [
        "https://stac.eoapi.dev",
        "https://stac.maap-project.org",
    ],
    "limit": 3,
}
response = httpx.get(
    f"{API_BASE_URL}/collections",
    params=params,
    timeout=None,
)
collections = response.json()

print(f"Searching specific APIs: {params['apis']}")
print(f"Collections returned: {len(collections['collections'])}")
print("\nSome collection IDs:")
for collection in collections["collections"]:
    print(f"- {collection['id']}")

# Check canonical links to see which APIs were actually queried
print("\nCanonical links (showing which APIs were queried):")
for link in collections["links"]:
    if link["rel"] == "canonical":
        print(f"- {link['href']}")

Searching specific APIs: ['https://stac.eoapi.dev', 'https://stac.maap-project.org']
Collections returned: 6

Some collection IDs:
- copernicus-dem
- MAXAR_afghanistan_earthquake22
- MAXAR_BayofBengal_Cyclone_Mocha_May_23
- ABoVE_UAVSAR_PALSAR
- AFRISAR_DLR
- AFRISAR_DLR2

Canonical links (showing which APIs were queried):
- https://stac.eoapi.dev/collections?limit=3&filter_lang=cql2-text
- https://stac.maap-project.org/collections?limit=3&filter_lang=cql2-text


## Advanced Searches

### Free-Text Search

Search collections using keywords (if supported by the upstream APIs):

In [17]:
# Free-text search for Landsat collections
params = {"q": "landsat", "limit": 2}
response = httpx.get(f"{API_BASE_URL}/collections", params=params)
collections = response.json()

print(f"Collections matching '{params['q']}':")
print(f"Collections returned: {len(collections['collections'])}")
print("\nMatching collections:")
for collection in collections["collections"]:
    title = collection.get("title", "No title")
    description = (
        collection.get("description", "")[:100] + "..."
        if len(collection.get("description", "")) > 100
        else collection.get("description", "")
    )
    api_link = next(
        (link["href"] for link in collection["links"] if link["rel"] == "self"),
        None,
    )
    print(f"- {collection['id']}")
    print(f"  Title: {title}")
    print(f"  Upstream API link: {api_link}")
    print(f"  Description: {description}")
    print()

Collections matching 'landsat':
Collections returned: 4

Matching collections:
- glad-glclu2020-change-v2
  Title: GLAD: Net change of land cover and land use between 2000 and 2020
  Upstream API link: https://stac.maap-project.org/collections/glad-glclu2020-change-v2
  Description: The GLAD Global Land Cover and Land Use Change dataset quantifies changes in forest extent and heigh...

- glad-glclu2020-v2
  Title: GLAD: Annual maps of land cover and land use
  Upstream API link: https://stac.maap-project.org/collections/glad-glclu2020-v2
  Description: The GLAD Global Land Cover and Land Use Change dataset quantifies changes in forest extent and heigh...

- HLSL30_2.0
  Title: HLS Landsat Operational Land Imager Surface Reflectance and TOA Brightness Daily Global 30m v2.0
  Upstream API link: https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30_2.0
  Description: The Harmonized Landsat Sentinel-2 (HLS) project provides consistent surface reflectance (SR) and top...

- OPERA_L

### Bounding Box Search

Search for collections that intersect with a geographic area:

In [18]:
# Search collections intersecting with a bounding box (example: continental US)
params = {
    "bbox": "-125,25,-66,49",  # [west, south, east, north]
    "limit": 2,
}
response = httpx.get(f"{API_BASE_URL}/collections", params=params)
collections = response.json()

print(f"Collections intersecting bbox {params['bbox']}:")
print(f"Collections returned: {len(collections['collections'])}")
print("\nCollection details:")
for collection in collections["collections"]:
    spatial_extent = collection.get("extent", {}).get("spatial", {})
    bbox_extent = spatial_extent.get("bbox", [[]])[0] if spatial_extent else "N/A"
    api_link = next(
        (link["href"] for link in collection["links"] if link["rel"] == "self"),
        None,
    )
    print(f"- {collection['id']}")
    print(f"  Upstream API link: {api_link}")
    print(f"  Extent: {bbox_extent}")
    print()

Collections intersecting bbox -125,25,-66,49:
Collections returned: 6

Collection details:
- ESACCI_Biomass_L4_AGB_V4_100m
  Upstream API link: https://stac.maap-project.org/collections/ESACCI_Biomass_L4_AGB_V4_100m
  Extent: [-180.0, -90.0, 180.0, 90.0]

- GEDI_CalVal_Field_Data
  Upstream API link: https://stac.maap-project.org/collections/GEDI_CalVal_Field_Data
  Extent: [-156.0001973, -43.1031025, 155.7179351, 65.4300929]

- copernicus-dem
  Upstream API link: https://stac.eoapi.dev/collections/copernicus-dem
  Extent: [-180.0, -90.0, 180.0, 90.0]

- MAXAR_Hurricane_Fiona_9_19_2022
  Upstream API link: https://stac.eoapi.dev/collections/MAXAR_Hurricane_Fiona_9_19_2022
  Extent: [-71.187593, 17.954147, -63.439022, 44.773025]

- urn:ogc:def:EOP:VITO:VGT_S10_1
  Upstream API link: https://cmr.earthdata.nasa.gov/stac/FEDEO/collections/urn%3Aogc%3Adef%3AEOP%3AVITO%3AVGT_S10_1
  Extent: [-180, -56, 180, 75]

- 14c_of_soil_co2_from_ipy_itex_cross_site_comparison
  Upstream API link: https

## Using pystac-client

The Collection Discovery API is fully compatible with `pystac-client`, which provides a more pythonic interface:

In [22]:
# Connect to the API using pystac-client
client = pystac_client.Client.open(API_BASE_URL)

print("API Info:")
print(f"Title: {client.title}")
print(f"Description: {client.description[:100]}...")

# Get conformance classes
conformance = client.get_conforms_to()
print(f"\nConformance classes: {len(conformance)} classes")
for cls in conformance:
    print(f"- {cls}")

API Info:
Title: STAC Collection Discovery API
Description: A collection-search-only STAC API that combines paginated search results from multiple upstream STAC...

Conformance classes: 12 classes
- http://www.opengis.net/spec/cql2/1.0/conf/basic-cql2
- http://www.opengis.net/spec/cql2/1.0/conf/cql2-json
- http://www.opengis.net/spec/cql2/1.0/conf/cql2-text
- http://www.opengis.net/spec/ogcapi-common-2/1.0/conf/simple-query
- http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/core
- http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/oas30
- http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/filter
- https://api.stacspec.org/v1.0.0-rc.1/collection-search
- https://api.stacspec.org/v1.0.0-rc.1/collection-search#free-text
- https://api.stacspec.org/v1.0.0-rc.1/collection-search#sort
- https://api.stacspec.org/v1.0.0/collections
- https://api.stacspec.org/v1.0.0/core


In [23]:
# Search for biomass collections using pystac-client
search = client.collection_search(q="biomass", limit=2)

print("Searching for biomass collections using pystac-client:")

# Show collection details
for i, collection in enumerate(search.collections()):
    print(f"\n- {collection.id}")
    print(f"  Title: {collection.title}")
    print(f"  Upstream API link: {collection.get_self_href()}")
    description = (
        collection.description[:100] + "..."
        if len(collection.description) > 100
        else collection.description
    )
    print(f"  Description: {description}")
    if i > 3:
        break

Searching for biomass collections using pystac-client:

- AFRISAR_DLR
  Title: AFRISAR_DLR
  Upstream API link: https://stac.maap-project.org/collections/AFRISAR_DLR
  Description: The  ESA  BIOMASS  mission  was  selected  in  2013  as  the  7th  Earth  Explorer  mission.  BIOMAS...

- AFRISAR_DLR2
  Title: AFRISAR_DLR2
  Upstream API link: https://stac.maap-project.org/collections/AFRISAR_DLR2
  Description: The ESA BIOMASS mission was selected in 2013 as the 7th Earth Explorer mission. BIOMASS will provide...

- BIOMASS
  Title: Biological Investigations of Marine Antarctic Systems and Stocks (BIOMASS) Data Set
  Upstream API link: https://cmr.earthdata.nasa.gov/stac/SCIOPS/collections/BIOMASS
  Description: BIOMASS (Biological Investigations of Marine Antarctic Systems and Stocks) was
a major multi-nationa...

- 95913ffb6467447ca72c4e9d8cf30501_NA
  Title: ESA Biomass Climate Change Initiative (Biomass_cci): Global datasets of forest above-ground biomass for the years 2007, 2010, 2

### Searching Specific APIs with pystac-client

You can also specify which upstream APIs to query by supplying the `apis` parameter in a `request_modifier` function:

In [25]:
# Connect to specific APIs using pystac-client
def add_apis(request: Request):
    """Add the `apis` parameter to all requests to this STAC API client"""
    request.params.update(
        {
            "apis": [
                "https://stac.eoapi.dev",
                "https://stac.maap-project.org",
            ]
        }
    )
    return request


specific_client = pystac_client.Client.open(API_BASE_URL, request_modifier=add_apis)

# Search for biomass collections from specific APIs only
search = specific_client.collection_search(q="biomass", limit=2)

print("Searching specific APIs for biomass collections:")

# Show collection details
for i, collection in enumerate(search.collections()):
    print(f"\n- {collection.id}")
    print(f"  Title: {collection.title}")
    print(f"  Upstream API link: {collection.get_self_href()}")
    description = (
        collection.description[:100] + "..."
        if len(collection.description) > 100
        else collection.description
    )
    print(f"  Description: {description}")
    if i > 3:
        break

Searching specific APIs for biomass collections:

- AFRISAR_DLR
  Title: AFRISAR_DLR
  Upstream API link: https://stac.maap-project.org/collections/AFRISAR_DLR
  Description: The  ESA  BIOMASS  mission  was  selected  in  2013  as  the  7th  Earth  Explorer  mission.  BIOMAS...

- AFRISAR_DLR2
  Title: AFRISAR_DLR2
  Upstream API link: https://stac.maap-project.org/collections/AFRISAR_DLR2
  Description: The ESA BIOMASS mission was selected in 2013 as the 7th Earth Explorer mission. BIOMASS will provide...

- AfriSAR_UAVSAR_Geocoded_Covariance
  Title: AfriSAR UAVSAR Geocoded Covariance Matrix product Generated Using NISAR Tools
  Upstream API link: https://stac.maap-project.org/collections/AfriSAR_UAVSAR_Geocoded_Covariance
  Description: The Geocoded Covariance Matrix dataset is the 4x4 Native Covariance Matrix geocoded to a spatial res...

- AfriSAR_UAVSAR_Geocoded_SLC
  Title: AfriSAR UAVSAR Geocoded SLCs Generated Using NISAR Tools
  Upstream API link: https://stac.maap-project.or